In [2]:
import numpy as np

In [7]:
# Task 1_HW
# Find the roots of square equation by gradient descent
# x ** 2 - 6 * x + 4 = 0
import numpy as np

def grad_f(x):
  return 2*x - 6

def f(x):
  return x**2 - 6*x + 4

def grad_descent(x_init, learning_rate, precision):
    x_old = x_init
    x_new = x_old - learning_rate*grad_f(x_old)
    #print(x_old, x_new, f(x_new))
    while f(x_new) > precision:
        x_old = x_new
        x_new = x_old - learning_rate*grad_f(x_old)
        #print(x_old, x_new, f(x_new))
    return x_new

root1 = grad_descent(6, 0.001, 0.0001)
root2 = grad_descent(-2, 0.001, 0.0001)
print('x1 =', root1)
print('x2 =', root2)

# Если мы не возьмем очень маленький learning_rate, то сойдется за достойное количество шагов
# По-хорошему начальную точку тоже важно брать адекватно и не очень большую. 
# Выбор точки является важным действией, когда у вас несколько экстремумов у функции и вы плохим выбором точки можете в итоге попасть в локальный минимум
# Можно брать с двух разных сторон от центра, ли бо искать центр обычным градиентным спуском с условием abs(x_new - x_old) > precision
# А после брать x_init длz x1, как random(-inf; xцентр) и x2 = random (xцентр; +inf)
# Можно подставить любые a и b и с, => производная от этой штуки ax + b и делаем то же самое


x1 = 5.235171359579289
x2 = 0.7641245785548506


In [ ]:
# Task 2 _ HW

import torch

# Будем использовать активацию с помощью сигмоиды
# Можно подставить любую функцию активации, так как это было написано на семинаре
# Так как мы используем pytorch меняем метод dot на matmul
# Не стал использовать функции написанные на занятии sigmoid_backward, написал сам


class FullyConnectedLayer:
    def __init__(self, input_size, output_size):
        self.weights = torch.randn(output_size, input_size, requires_grad=True)
        self.bias = torch.randn(output_size, requires_grad=True)
        self.grad_weights = None
        self.grad_bias = None

    def forward(self, input):
        return torch.add(torch.matmul(self.weights, input), self.bias)

    def backward(self, input, grad_output):
        forw = self.forward(input)
        grad_lin = grad_output * torch.sigmoid(forw) * (1 - torch.sigmoid(forw))
        grad_input = torch.matmul(self.weights.t(), grad_lin)
        self.grad_weights = torch.matmul(grad_lin, input.t())
        self.grad_bias = torch.sum(grad_lin, dim=0)
        return grad_input



In [4]:
# Task 3_HW

# Написать 2 оптимизатора адаптивных

import math

class ADAgrad:
    def __init__(self, model: Model, lr= 0.0001):
        self.model = model
        self.lr = lr
        self.accumulated_w = None
        self.accumulated_b = None
        
    def step(self):
        self.accumulated_w += self.model.grad_weights ** 2
        adapt_lr = self.lr / math.sqrt(self.accumulated_w)
        self.model.weights = self.model.weights - adapt_lr * self.model.grad_weights
        
        self.accumulated_b += self.model.grad_bias ** 2
        adapt_lr = self.lr / math.sqrt(self.accumulated_b)
        self.model.bias = self.model.bias - adapt_lr * self.model.grad_bias
    
    def zero_grad(self):
        self.model._clear_state()

In [5]:

import math

class RMSProp:
    def __init__(self, model: Model, lr= 0.0001, rho = 0.9):
        self.model = model
        self.lr = lr
        self.rho = 0.9
        self.accumulated_w = None
        self.accumulated_b = None
        
    def step(self):
        self.accumulated_w += self.rho * self.accumulated_w + (1 - self.rho) * (self.model.grad_weights) ** 2
        adapt_lr = self.lr / math.sqrt(self.accumulated_w)
        self.model.weights = self.model.weights - adapt_lr * self.model.grad_weights
        
        self.accumulated_b = self.rho * self.accumulated_b + (1 - self.rho) * self.model.grad_bias ** 2
        adapt_lr = self.lr / math.sqrt(self.accumulated_b)
        self.model.bias = self.model.bias - adapt_lr * self.model.grad_bias
    
    def zero_grad(self):
        self.model._clear_state()

In [ ]:
# Task 2
# Realize forward and backward pass 

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def sigmoid_backward(da, x):
    sig = sigmoid(x)
    
    return da * sig * (1 - sig)

def relu(x):
    return np.maximum(0., x)

def relu_backward(da, x):
    da = np.array(da, copy = True)
    da[x <= 0] = 0;
    return da;

In [ ]:
def mse_loss(t, y):
    return (t - y) ** 2

def d_mse_loss(t, y):
    return 2 * (y - t) 


In [ ]:
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        self.w = np.random.randn(n_out, n_inp) * 0.1
        self.b = np.random.randn(n_out, 1) * 0.1
        if activation == 'sigmoid':
            self.activ = sigmoid
        if activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        self.lin = np.dot(self.w, x) + self.b
        activ = self.activ(self.lin) if self.activ is not None else self.lin

        return activ

    def backward(self, grad): # grad = d L / d z    Dout 
        # grad * dz / d lin
        if self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin) 
        elif self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        # grad_lin * d lin / d w 
        m = self.inp.shape[1]
        self.d_w = np.dot(grad_lin, self.inp.T) / m    # d_in dOut
        # grad_lin * d lin / d b 
        self.d_b = np.sum(grad_lin, axis=1, keepdims=True) / m

        grad = np.dot(self.w.T, grad_lin)

        return grad

# pred = model(x)
# loss = criterion(pred, target)
# grad = d loss / d pred
# model.backward(grad)

In [2]:
from typing import Tuple

class Model:
    def __init__(self, arch: Tuple[Tuple[int, int]], activation):
        self.layers = []
        for i, p in enumerate(arch):
            self.layers.append(
                LinearLayer(p[0], p[1], 
                            activation=activation if i < len(arch)-1 else 'None')
                )
        self._clear_state()
    
    def _clear_state(self):
        for l in self.layers:
            l._clear_state()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        
        return x

    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

        return grad 

In [ ]:
# Task 3
# Realize SGD Momentum optimizer
# velocity = momentum * velocity - lr * gradient
# w = w + velocity

In [ ]:
#для одного слоя
class SGDMomentum:
    def __init__(self, model: LinearLayer, lr=0.001, momentum=0.99):
        self.lr = lr
        self.m = momentum
        self.model = model

        self.vel_w = np.zeros_like(model.w)
        self.vel_b = np.zeros_like(model.b)

    def step(self):
        self.vel_w = self.m * self.vel_w - self.lr * self.model.d_w
        self.vel_b = self.m * self.vel_b - self.lr * self.model.d_b

        self.model.w += self.vel_w
        self.model.b += self.vel_b

        




In [ ]:
#для всей модели
class SGDMomentum:
    def __init__(self, model: Model, lr= 0.0001, momentum=0.0):
        self.model = model
        self.lr = lr
        self.m = momentum
        self.vel = [[np.zeros_like(layer.w), 
                     np.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.vel[i][0] = self.vel[i][0] * self.m - self.lr * layer.d_w 
            self.vel[i][1] = self.vel[i][1] * self.m - self.lr * layer.d_b 
            layer.w += self.vel[i][0]
            layer.b += self.vel[i][1]
    
    def zero_grad(self):
        self.model._clear_state()

In [ ]:
# pred = model(x)
# loss = criterion(pred, target)
# grad = d loss / d pred
# model.backward(grad)
# optim.step()

In [ ]:
x = np.random.uniform(-2, 2, 20000)
y = x**2 + np.random.randn()*0.1


In [ ]:
model = Model(((1, 100), (100, 1)), activation='relu')
optim = SGDMomentum(model)
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(np.array([[val]]))
        loss = mse_loss(t, pred)
        grad = d_mse_loss(t, pred)
        model.backward(grad)
        optim.step()
        
    print(e, model.forward([[1]]), model.forward([[2]]), model.forward([[-1]]), model.forward([[-2]]))